In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
from googleapiclient.discovery import build

api_key = 'YOUR_API_KEY'
youtube = build('youtube', 'v3', developerKey=api_key)
videos = []
nextPageToken = None


In [ ]:
# Create category dict
vid_cat_request = youtube.videoCategories().list(
        part="snippet",
        regionCode='VN'
    )

vid_cat_response = vid_cat_request.execute()
dict_categories = {}
for element in vid_cat_response['items']:
    dict_categories[element['id']] = element['snippet']['title']

In [ ]:
# Get videos id from channel
# UUabsTV34JwALXKGMqHpvUiA VTV24
# UUQ4JPrrur8XOuxNugXmc39g VTV4 news

video_ids_from_channel = []
i = 0
nextPageToken = None

while True:
    pl_request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId='UUQ4JPrrur8XOuxNugXmc39g',
        maxResults=50,
        pageToken=nextPageToken
    )
    pl_response = pl_request.execute()

    for item in pl_response['items']:
        video_ids_from_channel.append(item['contentDetails']['videoId'])

    nextPageToken = pl_response.get('nextPageToken')
    
    i += 1
    if i == 200:
        break
    print(i)

In [ ]:
# Extract info
likes = []
dislikes = []
views = []
titles = []
category_ids = []
dates = []
tags = []
comments = []

cols = [likes, dislikes, views, titles, category_ids, dates, tags, comments]
i = 0

for video_id in video_ids_from_channel:
    i += 1
    print(i)
    print(video_id)

    vid_request = youtube.videos().list(
                  part="statistics, snippet",
                  id=video_id
                  )
    vid_response = vid_request.execute()

    if vid_response['items'] != []:
        statistics = vid_response['items'][0]['statistics']
        snippet = vid_response['items'][0]['snippet']

        views.append(statistics['viewCount'])
        titles.append(snippet['localized']['title'])
        dates.append(snippet['publishedAt'])
        
        if 'likeCount' in statistics:
            likes.append(statistics['likeCount'])
        else:
            likes.append(np.nan)

        if 'dislikeCount' in statistics:
            dislikes.append(statistics['dislikeCount'])
        else:
            dislikes.append(np.nan)

        try:
            category_ids.append(dict_categories[snippet['categoryId']])
        except:
            category_ids.append(np.nan)

        if 'tags' in snippet:
            tags.append(snippet['tags'])
        else:
            tags.append(np.nan)
            
        if 'commentCount' in statistics:
            comments.append(statistics['commentCount'])
        else:
            comments.append(0)

    else:
        for col in cols: 
            col.append(np.nan)

In [ ]:
final_df = pd.DataFrame(data={'Likes': likes,
                              'Dislikes': dislikes,
                              'Views': views,
                              'Comments': comments,
                              'Title': titles,
                              'Category_id': category_ids,
                              'Date': dates,
                              'Tags': tags,
                              })
final_df

,Likes,Dislikes,Views,Comments,Title,Category_id,Date,Tags
0,0,0,14,0,VTV News 8h - 30/08/2021| VTV4,People & Blogs,2021-08-30T03:20:31Z,"[vietnam, vtv, vtv4, travel, Việt Nam]"
1,28,3,7027,1,Bản tin thời sự tiếng Việt 21h - 29/08/2021| VTV4,People & Blogs,2021-08-29T14:59:11Z,"[vietnam, vtv, vtv4, travel, Việt Nam]"
2,10,2,1102,0,Tình nguyện viên Việt Nam tại Thế vận hội |202...,People & Blogs,2021-08-29T12:31:24Z,"[vietnam, vtv, vtv4, travel, Việt Nam]"
3,20,3,914,0,Đường dây nóng kết nối tình quân dân | VTV4,People & Blogs,2021-08-29T11:26:59Z,"[vietnam, vtv, vtv4, travel, Việt Nam]"
4,0,0,2520,0,Chàng trai Ucraina và tình yêu với thanh âm củ...,People & Blogs,2021-08-29T09:56:57Z,"[vietnam, vtv, vtv4, travel, Việt Nam]"
...,...,...,...,...,...,...,...,...
9626,6,0,531,0,VTV News 8h - 06/03/2019,People & Blogs,2019-03-06T06:33:15Z,NaN
9627,2,1,93,0,Программы на русском языке - 05/03/2019,People & Blogs,2019-03-06T06:00:54Z,NaN
9628,6,1,264,0,今日越南 05/03/2019,People & Blogs,2019-03-05T14:53:45Z,NaN
9629,6,0,973,0,Bản tin thời sự tiếng Việt 12h - 05/03/2019,People & Blogs,2019-03-05T10:15:10Z,NaN


In [ ]:
final_df.to_csv('VTV4.csv', index=False)